In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as spark_sum, mean, min, max


In [2]:
spark = SparkSession.builder.appName("FlightData").getOrCreate()

25/10/01 13:25:04 WARN Utils: Your hostname, mac resolves to a loopback address: 127.0.1.1; using 192.168.100.5 instead (on interface wlo1)
25/10/01 13:25:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/01 13:25:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/10/01 13:25:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
#load data
df = spark.read.parquet("../data/raw/flight_data.parquet")
df.show(5)

25/10/01 13:26:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+---------------------------------------+----------------------------------------------+-------------------------------------------------+--------------------------------------------------+------------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+--------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+-----------

In [5]:
df.printSchema()

root
 |-- Year: long (nullable = true)
 |-- Quarter: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- DayofMonth: long (nullable = true)
 |-- DayOfWeek: long (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |-- DOT_ID_Marketing_Airline: long (nullable = true)
 |-- IATA_Code_Marketing_Airline: string (nullable = true)
 |-- Flight_Number_Marketing_Airline: long (nullable = true)
 |-- Originally_Scheduled_Code_Share_Airline: string (nullable = true)
 |-- DOT_ID_Originally_Scheduled_Code_Share_Airline: double (nullable = true)
 |-- IATA_Code_Originally_Scheduled_Code_Share_Airline: string (nullable = true)
 |-- Flight_Num_Originally_Scheduled_Code_Share_Airline: double (nullable = true)
 |-- Operating_Airline : string (nullable = true)
 |-- DOT_ID_Operating_Airline: long (nullable = true)
 |-- IATA_Code_Operating_Airline: string (nullable

In [6]:
print(f"Row count: {df.count()}")
print(f"Column count: {len(df.columns)}")

Row count: 582425
Column count: 120


In [7]:
# print columns
for c in df.columns:
    print("-", c)

- Year
- Quarter
- Month
- DayofMonth
- DayOfWeek
- FlightDate
- Marketing_Airline_Network
- Operated_or_Branded_Code_Share_Partners
- DOT_ID_Marketing_Airline
- IATA_Code_Marketing_Airline
- Flight_Number_Marketing_Airline
- Originally_Scheduled_Code_Share_Airline
- DOT_ID_Originally_Scheduled_Code_Share_Airline
- IATA_Code_Originally_Scheduled_Code_Share_Airline
- Flight_Num_Originally_Scheduled_Code_Share_Airline
- Operating_Airline 
- DOT_ID_Operating_Airline
- IATA_Code_Operating_Airline
- Tail_Number
- Flight_Number_Operating_Airline
- OriginAirportID
- OriginAirportSeqID
- OriginCityMarketID
- Origin
- OriginCityName
- OriginState
- OriginStateFips
- OriginStateName
- OriginWac
- DestAirportID
- DestAirportSeqID
- DestCityMarketID
- Dest
- DestCityName
- DestState
- DestStateFips
- DestStateName
- DestWac
- CRSDepTime
- DepTime
- DepDelay
- DepDelayMinutes
- DepDel15
- DepartureDelayGroups
- DepTimeBlk
- TaxiOut
- WheelsOff
- WheelsOn
- TaxiIn
- CRSArrTime
- ArrTime
- ArrDelay
-

In [8]:
# check for null
null_counts = df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])
null_counts.show(truncate=False)

+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+---------------------------------------+----------------------------------------------+-------------------------------------------------+--------------------------------------------------+------------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+-------------

In [9]:
nulls_dict = null_counts.first().asDict()

# filter columns with > 0 nulls
cols_with_nulls = {col: count for col, count in nulls_dict.items() if count>0}

print(f"Columns with nulls: {len(cols_with_nulls)}")

Columns with nulls: 75


In [15]:
df_nulls = null_counts.select(*cols_with_nulls)
df_nulls.show(truncate=False)

+---------------------------------------+----------------------------------------------+-------------------------------------------------+--------------------------------------------------+-----------+-------+--------+---------------+--------+--------------------+-------+---------+--------+------+-------+--------+---------------+--------+------------------+----------------+-----------------+-------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+----------------+------------+--------------+---------------

In [16]:
len(df_nulls.columns)

75

In [18]:
# keep columns with no nulls
cols_with_no_nulls = [c for c, cnt in nulls_dict.items() if cnt ==0]
df_no_nulls = df.select(*cols_with_no_nulls)
df_no_nulls.show(5)

+----+-------+-----+----------+---------+----------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+------------------+------------------------+---------------------------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+--------------+---------+-------------+-------------+-------+----------+----------+----------+----------+---------+--------+--------------+-------+--------+-------------+------------------+---------+
|Year|Quarter|Month|DayofMonth|DayOfWeek|FlightDate|Marketing_Airline_Network|Operated_or_Branded_Code_Share_Partners|DOT_ID_Marketing_Airline|IATA_Code_Marketing_Airline|Flight_Number_Marketing_Airline|Operating_Airline |DOT_ID_Operating_Airline|IATA_Code_Operating_Airline|Flight_Number_Operating_Airline|Ori

In [19]:
len(df_no_nulls.columns)

45

In [12]:
df.groupBy("Marketing_Airline_Network").agg(mean("DepDelay"), mean("ArrDelay")).show()

+-------------------------+------------------+------------------+
|Marketing_Airline_Network|     avg(DepDelay)|     avg(ArrDelay)|
+-------------------------+------------------+------------------+
|                       UA|16.921039429092136|10.146746730278851|
|                       NK|15.046109510086456| 9.684378268014541|
|                       AA| 19.63141945439438|16.279486369855803|
|                       B6|19.153642452879172|13.522219904026706|
|                       DL|14.515689176526225| 7.575911260338751|
|                       F9|18.852062184993255|14.719330722677109|
|                       HA| 9.958526056120876|10.664710424710425|
|                       G4|15.406684905435947|10.528209419077218|
|                       AS|13.577607050932064| 9.952008182211557|
|                       WN| 12.14088938547486| 6.420730560060208|
+-------------------------+------------------+------------------+



In [14]:
df.groupBy("Origin", "Dest").count().orderBy("count", ascending=False).show()

+------+----+-----+
|Origin|Dest|count|
+------+----+-----+
|   OGG| HNL| 1001|
|   HNL| OGG|  994|
|   LAX| SFO|  858|
|   SFO| LAX|  857|
|   PHX| DEN|  832|
|   DEN| PHX|  831|
|   ATL| MCO|  792|
|   MCO| ATL|  792|
|   ORD| LGA|  789|
|   LGA| ORD|  789|
|   LAX| LAS|  783|
|   LAS| LAX|  782|
|   LAX| JFK|  752|
|   JFK| LAX|  752|
|   ATL| FLL|  732|
|   FLL| ATL|  731|
|   BOS| DCA|  731|
|   DCA| BOS|  731|
|   HNL| LIH|  730|
|   LIH| HNL|  730|
+------+----+-----+
only showing top 20 rows



In [21]:
# top busiest routes
df.groupBy("Origin", "Dest").count().orderBy(col("Count").desc()).show(10, truncate=False)

+------+----+-----+
|Origin|Dest|count|
+------+----+-----+
|OGG   |HNL |1001 |
|HNL   |OGG |994  |
|LAX   |SFO |858  |
|SFO   |LAX |857  |
|PHX   |DEN |832  |
|DEN   |PHX |831  |
|ATL   |MCO |792  |
|MCO   |ATL |792  |
|ORD   |LGA |789  |
|LGA   |ORD |789  |
+------+----+-----+
only showing top 10 rows



In [22]:
# Filter by marketing line
df.groupBy("Marketing_Airline_Network")\
    .count()\
    .orderBy(col("Count").desc())\
    .show(10, truncate=False)

+-------------------------+------+
|Marketing_Airline_Network|count |
+-------------------------+------+
|AA                       |147443|
|DL                       |116199|
|WN                       |115389|
|UA                       |104791|
|AS                       |29057 |
|NK                       |20415 |
|B6                       |19580 |
|F9                       |14379 |
|G4                       |8596  |
|HA                       |6576  |
+-------------------------+------+



In [26]:
# Top 10 airports
df.groupBy("Origin")\
    .count()\
    .orderBy(col("Count").desc())\
    .show(10, truncate=False)

+------+-----+
|Origin|count|
+------+-----+
|ATL   |26363|
|DEN   |24500|
|DFW   |23743|
|ORD   |22999|
|CLT   |19126|
|PHX   |15573|
|LAX   |15341|
|LAS   |15053|
|MCO   |14296|
|SEA   |13140|
+------+-----+
only showing top 10 rows



In [27]:
# Avg delay per airline
df.groupBy("Marketing_Airline_Network") \
  .agg({"DepDelay": "avg", "ArrDelay": "avg"}) \
  .orderBy(col("avg(DepDelay)").desc()) \
  .show(10, truncate=False)

# Avg delay per Origin airport
df.groupBy("Origin") \
  .agg({"DepDelay": "avg"}) \
  .orderBy(col("avg(DepDelay)").desc()) \
  .show(10, truncate=False)


+-------------------------+------------------+------------------+
|Marketing_Airline_Network|avg(DepDelay)     |avg(ArrDelay)     |
+-------------------------+------------------+------------------+
|AA                       |19.63141945439438 |16.279486369855803|
|B6                       |19.153642452879172|13.522219904026706|
|F9                       |18.852062184993255|14.719330722677109|
|UA                       |16.921039429092136|10.146746730278851|
|G4                       |15.406684905435947|10.528209419077218|
|NK                       |15.046109510086456|9.684378268014541 |
|DL                       |14.515689176526225|7.575911260338751 |
|AS                       |13.577607050932064|9.952008182211557 |
|WN                       |12.14088938547486 |6.420730560060208 |
|HA                       |9.958526056120876 |10.664710424710425|
+-------------------------+------------------+------------------+

+------+------------------+
|Origin|avg(DepDelay)     |
+------+-----------

25/10/02 09:57:34 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 31831999 ms exceeds timeout 120000 ms
25/10/02 09:57:34 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/02 09:57:41 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at